In [2]:
!pip install TA-lib

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 368.5/368.5 kB 25.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  error: subprocess-exited-with-error
  
  × Building wheel for TA-lib (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> [21 lines of output]
      <string>:75: UserWarning: Cannot find ta-lib library, installation may fail.
      running bdist_wheel
      running build
      running build_py
      creating build
      creating build/lib.linux-x86_64-cpython-310
      creating build/lib.linux-x86_64-cpython-310/talib
      copying talib/__init__.py -> build/lib.linux-x86_64-cpython-310/talib
      copying talib/abstract.py -> build/lib.linux-x86_64-cpython-310/talib
      copying talib/deprecated.py -> build/lib.linux-x86_64-cpython-310/talib
      copying talib/stream.py -> build/lib.linux-x86_64-cpython-310/talib
      running build_ext
      building 'tal

In [1]:
from KAN import KAN
import pandas as pd
import talib
import numpy as np

data = pd.read_csv('600519.SH.CSV', encoding='gbk')
data.drop(columns=['Unnamed: 18'], inplace=True)

predict_period = 21

# construct factors
data['return1'] = data['收盘价(元)'].pct_change()
data['rv_product'] = data['return1'] * data['成交量(股)']  # 交易量与日收益乘积
data['high_r'] = data['return1'] * data['最高价(元)'] / data['收盘价(元)']  # 今天最高价除以昨天收盘价
data['low_r'] = data['return1'] * data['最低价(元)'] / data['收盘价(元)']  # 今天最低价除以昨天收盘价
data['hml_r'] = data['high_r'] - data['low_r']  # high minus low
data['hpl_r'] = data['high_r'] + data['low_r']  # high plus low
data['over_night_mom'] = data['开盘价(元)'] / data['收盘价(元)'].shift(1)
data['intraday_mom'] = data['收盘价(元)'] / data['开盘价(元)']
for N in [1, 2, 6, 12]:
    # 动量因子
    # return_Nm, N = 1, 2, 6, 12 近N个月的收益率
    # wgt_return_Nm, N = 1, 2, 6, 12 以每日交易量为权重对每日收益率求加权平均
    data[f'return_{N}m'] = data['收盘价(元)'].pct_change(periods=N * 21)
    data[f'wgt_return_{N}m'] = data['rv_product'].rolling(N * 21).sum() / data['成交量(股)'].rolling(N * 21).sum()
    # 波动率因子
    data[f'high_r_std_{N}m'] = data['high_r'].rolling(N * 21).std()
    data[f'low_r_std_{N}m'] = data['low_r'].rolling(N * 21).std()
    data[f'hml_r_std_{N}m'] = data['hml_r'].rolling(N * 21).std()
    data[f'hpl_r_std_{N}m'] = data['hpl_r'].rolling(N * 21).std()
for N in [1, 2]:
    # 平滑动量
    data[f'smomemtum_{N}m'] = data['收盘价(元)'].pct_change(N * 21) / data['return1'].rolling(N * 21).apply(
        lambda x: np.sum(np.abs(x)))
    # 隔夜收益率与日内收益率之和
    data[f'over_night_mom_{N}m'] = np.log(data['over_night_mom']).rolling(N * 21).sum()
    data[f'intraday_mom_{N}m'] = np.log(data['intraday_mom']).rolling(N * 21).sum()
# 52周最高价距离动量
data['high250'] = data['收盘价(元)'] / data['收盘价(元)'].rolling(250).max() - 1
data['return5_label'] = data['收盘价(元)'].pct_change(predict_period).shift(-predict_period)
data['RSI'] = talib.RSI(data['收盘价(元)'])
data['obv'] = talib.OBV(data['收盘价(元)'], data['成交量(股)'])
data.drop(columns=['return1', 'rv_product', 'high_r', 'low_r', 'hml_r', 'hpl_r', '成交量(股)'],
          inplace=True)
data.dropna(inplace=True)
data.reset_index(drop=True, inplace=True)
data

,代码,简称,日期,开盘价(元),最高价(元),最低价(元),收盘价(元),成交金额(元),涨跌(元),涨跌幅(%),...,smomemtum_1m,over_night_mom_1m,intraday_mom_1m,smomemtum_2m,over_night_mom_2m,intraday_mom_2m,high250,return5_label,RSI,obv
0,600519.SH,贵州茅台,2005-06-13,76.6695,78.0516,75.6938,76.7996,3.338907e+07,0.3415,0.4466,...,-0.061076,-0.069087,0.038028,-0.066419,-0.097969,0.039377,-0.108867,0.132755,50.883298,90470435.0
1,600519.SH,贵州茅台,2005-06-14,76.4256,78.8159,76.4256,77.5150,3.453065e+07,0.7155,0.9316,...,-0.049677,-0.069031,0.043556,-0.051813,-0.102850,0.057131,-0.100566,0.101322,52.299676,91197584.0
2,600519.SH,贵州茅台,2005-06-15,76.6044,78.0516,76.5882,77.4500,7.599370e+06,-0.0650,-0.0839,...,-0.082719,-0.082891,0.041355,-0.014260,-0.112662,0.100765,-0.101320,0.110644,52.152521,91037907.0
3,600519.SH,贵州茅台,2005-06-16,77.5638,78.8809,77.2386,78.8484,2.845240e+07,1.3984,1.8056,...,0.021417,-0.086244,0.096401,0.023182,-0.111608,0.130763,-0.085094,0.092803,55.080786,91631060.0
4,600519.SH,贵州茅台,2005-06-17,78.8647,79.6777,77.4337,78.8647,1.570394e+07,0.0163,0.0206,...,-0.030114,-0.086038,0.072318,-0.009888,-0.126631,0.118622,-0.084905,0.086598,55.115268,91957359.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2443,600519.SH,贵州茅台,2023-12-25,13130.2334,13189.9520,13106.6603,13138.0911,2.086014e+09,10.6079,0.0808,...,-0.276313,-0.020666,-0.028765,0.033326,0.066365,-0.055028,-0.108437,-0.027512,44.947427,812714477.0
2444,600519.SH,贵州茅台,2023-12-26,13142.0200,13160.8785,13024.1543,13122.3757,2.457177e+09,-15.7154,-0.1196,...,-0.249124,-0.018283,-0.024436,0.021996,0.071598,-0.064133,-0.109503,-0.019162,44.429946,811237351.0
2445,600519.SH,贵州茅台,2023-12-27,13106.6603,13178.5583,13051.6563,13099.2740,2.679221e+09,-23.1017,-0.1760,...,-0.281420,-0.018346,-0.029555,0.009276,0.075008,-0.071861,-0.111071,-0.017840,43.634659,809631801.0
2446,600519.SH,贵州茅台,2023-12-28,13130.1548,13578.0443,13099.2740,13554.4712,6.537401e+09,455.1971,3.4750,...,-0.084951,-0.015403,-0.001538,0.096134,0.077958,-0.042779,-0.080181,-0.043473,59.150499,813465607.0
